<a href="https://colab.research.google.com/github/analyticswithadam/Python/blob/main/Lead_Allocation_Big_Query_%26_GPT4o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install openai
!pip install tabulate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [1]:
# get the openai secret key
import pandas as pd
import getpass
secret_key = getpass.getpass('Please enter your openai key: ')
from IPython.display import display, Markdown
from tabulate import tabulate

Please enter your openai key: ··········


In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
%%bigquery staff --project bigquerycourse-onetwothree
SELECT * FROM `bigquerycourse-onetwothree.test.Staff`

Query is running:   0%|          |

Downloading:   0%|          |

In [4]:
staff

,ID,Name,Sales New BMW,Sales New Nissan,Sales Used Premium,Sales Used Other,Bio,Current Leads
0,5,Tom Pops,0,30,0,12,Tom leads the Nissan new Sales with 10 years o...,10
1,2,Tony Young,2,2,2,30,Tony is a first year salesman specialising in ...,5
2,1,Tim Green,23,1,10,4,Tim is an experieced Salesman specialising in ...,10
3,4,Brid Flynn,10,10,10,10,Brid has been with the dealership 20 years and...,7
4,3,Mary Choco,3,20,20,5,Mary focuses on Premium Used and New Nissan,8


In [58]:
%%bigquery leads --project bigquerycourse-onetwothree
SELECT * FROM `bigquerycourse-onetwothree.test.Leads`

Query is running:   0%|          |

Downloading:   0%|          |

In [59]:
leads

,No,Name,New Customer,Interested in,Enquiry,Allocated,Sales Person,allocation_reason
0,7,Jeff Stubbs,False,Used,"There is a 05 BMW 3 Series on the site, is it ...",False,None,None
1,9,Fred Flynn,False,Used,Im looking to trade in X-Trail I bought from B...,False,None,None
2,3,Arron Wise,False,Used,"Dear Dealership, Have you heard about the new ...",False,None,None
3,4,Mairead Bo,True,Used,£££$£@$£@$$%£%$@£$!@£$R£!,False,None,None
4,10,Kerry Gregg,True,Used,Can I arrange a test drive of the 20D Opel Ast...,False,None,None
5,8,Perry Terry,True,Used,Is the 23D black Jag Sold?,False,None,None
6,1,Dave Young,True,New BMW,Im interested in the new 5 Series changed by C...,False,None,None
7,6,Barry O Brien,True,New BMW,Looking for more info on the new 3 series,False,None,None
8,2,Joe Peats,False,New Nissan,"Hello, I like the look of the new X-Trail, ple...",False,None,None
9,5,Josh Tas,False,New Nissan,"Hi Guys, Do you have any avaliability of the N...",False,None,None


In [60]:
leads_markdown = tabulate(leads, headers='keys', tablefmt='pipe', showindex=False)
salespeople_markdown = tabulate(staff, headers='keys', tablefmt='pipe', showindex=False)

In [61]:
print(leads_markdown)

|   No | Name          | New Customer    | Interested in    | Enquiry                                                                                                                                                   | Allocated   | Sales Person   | allocation_reason   |
|-----:|:--------------|:----------------|:-----------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------|:------------|:---------------|:--------------------|
|    7 | Jeff Stubbs   | False           | Used             | There is a 05 BMW 3 Series on the site, is it sold?                                                                                                       | False       |                |                     |
|    9 | Fred Flynn    | False           | Used             | Im looking to trade in X-Trail I bought from Brid and upgrade to one of the 221D ones you have in stock, can you get her to g

In [62]:
from openai import OpenAI
client = OpenAI(api_key=secret_key)

def get_response(prompt):
  # Create a request to the chat completions endpoint
  response = client.chat.completions.create(
    model="gpt-4o",
    # Assign the role and content for the message
    messages=
     [
        {
            "role": "system", "content": f"""

              You are an AI assistant tasked with allocating leads to salespeople for my Car Dealership based on the following rules:
              1. Consider if any leads are spam - Allocate these to the Salesperson Spam
              2. Allocate leads to the correct salesperson based on  speciality while ensuring lead allocation is balanced between the salespeople considering current leads.


              For each lead, provide the allocation in the following format:
              [
                  {{"No": lead_no, "Name": lead_name, "Sales Person": salesperson, "allocation_reason": "reason for allocation"}},
                  ...
              ]

              Provide no description text just output the json within the brackets.

            """
        },
        {
            "role": "user", "content": prompt
        }
     ],
    temperature = 0.4,
    max_tokens=3000)
  return response.choices[0].message.content

In [63]:
# Extract the result
result = get_response(f"""Leads:
              {leads_markdown}
              Salespeople:
              {salespeople_markdown}""")

# Add Results to Big Query


In [64]:
print(result)

[
    {"No": 7, "Name": "Jeff Stubbs", "Sales Person": "Tony Young", "allocation_reason": "Used car enquiry, balanced allocation"},
    {"No": 9, "Name": "Fred Flynn", "Sales Person": "Brid Flynn", "allocation_reason": "Used car enquiry, balanced allocation"},
    {"No": 3, "Name": "Arron Wise", "Sales Person": "Spam", "allocation_reason": "Spam enquiry"},
    {"No": 4, "Name": "Mairead Bo", "Sales Person": "Spam", "allocation_reason": "Spam enquiry"},
    {"No": 10, "Name": "Kerry Gregg", "Sales Person": "Tony Young", "allocation_reason": "Used car enquiry, balanced allocation"},
    {"No": 8, "Name": "Perry Terry", "Sales Person": "Brid Flynn", "allocation_reason": "Used car enquiry, balanced allocation"},
    {"No": 1, "Name": "Dave Young", "Sales Person": "Tim Green", "allocation_reason": "New BMW enquiry, balanced allocation"},
    {"No": 6, "Name": "Barry O Brien", "Sales Person": "Tim Green", "allocation_reason": "New BMW enquiry, balanced allocation"},
    {"No": 2, "Name": "Jo

In [54]:
# Convert the output to a DataFrame
allocations_list = eval(result.strip())
allocations_df = pd.DataFrame(allocations_list)

# Function to generate the update code
def generate_update_code(row):
    return f"""
UPDATE `bigquerycourse-onetwothree.test.Leads`
SET Allocated = True, `Sales Person` = '{row['Sales Person']}', allocation_reason = '{row['allocation_reason']}'
WHERE `Name` = '{row['Name']}';
"""

# Generate the update code for each row
update_code = ""
for _, row in allocations_df.iterrows():
    update_code += generate_update_code(row)

print(update_code)



UPDATE `bigquerycourse-onetwothree.test.Leads`
SET Allocated = True, `Sales Person` = 'Tony Young', allocation_reason = 'Specialises in all used cars and has balanced lead numbers'
WHERE `Name` = 'Jeff Stubbs';

UPDATE `bigquerycourse-onetwothree.test.Leads`
SET Allocated = True, `Sales Person` = 'Brid Flynn', allocation_reason = 'Requested Brid specifically and has balanced lead numbers'
WHERE `Name` = 'Fred Flynn';

UPDATE `bigquerycourse-onetwothree.test.Leads`
SET Allocated = True, `Sales Person` = 'Spam', allocation_reason = 'Spam enquiry'
WHERE `Name` = 'Arron Wise';

UPDATE `bigquerycourse-onetwothree.test.Leads`
SET Allocated = True, `Sales Person` = 'Tom Pops', allocation_reason = 'Specialises in New Nissan and has balanced lead numbers'
WHERE `Name` = 'Josh Tas';

UPDATE `bigquerycourse-onetwothree.test.Leads`
SET Allocated = True, `Sales Person` = 'Tom Pops', allocation_reason = 'Specialises in New Nissan and has balanced lead numbers'
WHERE `Name` = 'Joe Peats';

UPDATE `b

In [55]:
allocations_df

,No,Name,Sales Person,allocation_reason
0,7,Jeff Stubbs,Tony Young,Specialises in all used cars and has balanced ...
1,9,Fred Flynn,Brid Flynn,Requested Brid specifically and has balanced l...
2,3,Arron Wise,Spam,Spam enquiry
3,5,Josh Tas,Tom Pops,Specialises in New Nissan and has balanced lea...
4,2,Joe Peats,Tom Pops,Specialises in New Nissan and has balanced lea...
5,10,Kerry Gregg,Tony Young,Specialises in all used cars and has balanced ...
6,8,Perry Terry,Tony Young,Specialises in all used cars and has balanced ...
7,4,Mairead Bo,Spam,Spam enquiry
8,1,Dave Young,Tim Green,Specialises in New BMW and has balanced lead n...
9,6,Barry O Brien,Tim Green,Specialises in New BMW and has balanced lead n...


In [56]:
from google.cloud import bigquery
# Initialize BigQuery client
client = bigquery.Client(project='bigquerycourse-onetwothree')

# Split the update_code string into individual queries
queries = update_code.strip().split(';')

# Execute each query
for query in queries:
    if query.strip():  # Ensure the query is not empty
        query = query.strip() + ';'  # Ensure the query ends with a semicolon
        print(f"Executing query: {query}")  # For debugging purposes
        client.query(query).result()

print("All updates executed successfully.")

Executing query: UPDATE `bigquerycourse-onetwothree.test.Leads`
SET Allocated = True, `Sales Person` = 'Tony Young', allocation_reason = 'Specialises in all used cars and has balanced lead numbers'
WHERE `Name` = 'Jeff Stubbs';
Executing query: UPDATE `bigquerycourse-onetwothree.test.Leads`
SET Allocated = True, `Sales Person` = 'Brid Flynn', allocation_reason = 'Requested Brid specifically and has balanced lead numbers'
WHERE `Name` = 'Fred Flynn';
Executing query: UPDATE `bigquerycourse-onetwothree.test.Leads`
SET Allocated = True, `Sales Person` = 'Spam', allocation_reason = 'Spam enquiry'
WHERE `Name` = 'Arron Wise';
Executing query: UPDATE `bigquerycourse-onetwothree.test.Leads`
SET Allocated = True, `Sales Person` = 'Tom Pops', allocation_reason = 'Specialises in New Nissan and has balanced lead numbers'
WHERE `Name` = 'Josh Tas';
Executing query: UPDATE `bigquerycourse-onetwothree.test.Leads`
SET Allocated = True, `Sales Person` = 'Tom Pops', allocation_reason = 'Specialises in 

## Big Query Magic Commands

In [ ]:
# Rename Table Column
%%bigquery --project bigquerycourse-onetwothree
ALTER TABLE `bigquerycourse-onetwothree.test.Leads`
RENAME COLUMN `Enquiry ` TO `Enquiry`;

In [ ]:
# Add Column to table
%%bigquery --project bigquerycourse-onetwothree
ALTER TABLE `bigquerycourse-onetwothree.test.Leads`
ADD COLUMN allocation_reason STRING;

In [ ]:
# Update Row
%%bigquery --project bigquerycourse-onetwothree
UPDATE `bigquerycourse-onetwothree.test.Leads`
SET `Sales Person` = 'New Sales Person 1',
    allocation_reason = 'Reason for allocation 1',
    Allocated = True
WHERE `Name` = 'Arron Wise';

UPDATE `bigquerycourse-onetwothree.test.Leads`
SET `Sales Person` = 'New Sales Person 2',
    allocation_reason = 'Reason for allocation 2',
    Allocated = True
WHERE `Name` = 'Jeff Stubbs';

In [57]:
#Reset Allocations
%%bigquery --project bigquerycourse-onetwothree

UPDATE `bigquerycourse-onetwothree.test.Leads`
SET Allocated = False , `Sales Person` = NULL, allocation_reason = NULL
Where `Enquiry` is not null;

Query is running:   0%|          |

""
